<a href="https://colab.research.google.com/github/41371131h-chi/project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import os
import time
import requests
import pandas as pd
import gspread
import google.generativeai as genai
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from google.colab import auth
from google.auth import default
from google.colab import userdata
from gspread_dataframe import set_with_dataframe
import gradio as gr

In [42]:
# =================設定區=================
# 請在這裡填入你的 Google Sheet 網址
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1Qqq0I4pwqWU4xeEQ91XmT2buqwzT5a1LhY4oaWD_YKY/edit?usp=sharing'
# 指定要寫入的工作表名稱
WORKSHEET_NAME = '工作表1'
# =======================================

In [43]:
# 1. Google 驗證與設定
print("正在進行 Google 驗證...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 設定 Gemini API Key (從 Colab Secrets 讀取)
try:
    api_key = userdata.get('HW3')
    genai.configure(api_key=api_key)
    print("✅ Gemini API 設定完成")
except Exception as e:
    print(f"❌ API Key 設定失敗: {e} (請確認 Colab Secrets 是否有名為 HW3 的金鑰)")

正在進行 Google 驗證...
✅ Gemini API 設定完成


In [44]:
# ---------------------------------------------------------
# 2. 爬蟲函式：指定抓取「一週預報」
# ---------------------------------------------------------
def download_weekly_weather_pdf():
    target_url = "https://www.cwa.gov.tw/V8/C/W/pdf.html"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    print(f"正在連線至 {target_url} 尋找1週預報...")

    try:
        response = requests.get(target_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # 找出頁面上所有 .pdf 的連結
        all_pdf_links = soup.select("a[href$='.pdf']")

        target_link = None
        # 遍歷連結，尋找文字包含「一週」的那個
        for link in all_pdf_links:
            if "1週" in link.text:
                target_link = link
                print(f"✅ 找到目標連結文字：{link.text.strip()}")
                break

        if target_link:
            full_pdf_url = urljoin(target_url, target_link.get('href'))
            file_name = "weekly_forecast.pdf"
            print(f"正在下載 PDF: {full_pdf_url}")

            pdf_response = requests.get(full_pdf_url, headers=headers, timeout=30)
            with open(file_name, 'wb') as f:
                f.write(pdf_response.content)
            return file_name
        else:
            print("❌ 找不到包含「1週」字眼的 PDF 連結")
            return None

    except Exception as e:
        print(f"❌ 爬蟲失敗: {e}")
        return None

In [45]:
# ---------------------------------------------------------
# 3. 主核心函式 (已修改：接收 selected_region 參數)
# ---------------------------------------------------------
def get_weekly_fashion_advice(selected_region):
    """
    爬取一週預報 -> AI 生成一週穿搭 -> 回傳 DataFrame
    """
    status_msg = f"🚀 開始執行 [{selected_region}] 的一週穿搭分析...\n"
    print(status_msg)

    # 1. 執行爬蟲
    pdf_filename = download_weekly_weather_pdf()

    if not pdf_filename or not os.path.exists(pdf_filename):
         return pd.DataFrame({"狀態": ["❌ 失敗"], "原因": ["無法下載一週預報 PDF"]})

    status_msg += "✅ PDF 下載成功，AI 正在規劃一週穿搭...\n"

    try:
        # 2. 上傳檔案到 Gemini
        gemini_file = genai.upload_file(path=pdf_filename, display_name="Weekly Weather PDF")

        # ==============================================================================
        # 🔥 Prompt 修改：動態插入使用者選擇的地區 🔥
        # ==============================================================================
        prompt_text = f"""
        你是一位頂尖的時尚造型師。請閱讀這份「一週天氣預報」PDF。

        任務：請根據預報內容，為我規劃「{selected_region}」接下來 7 天的時尚穿搭行事曆。

        請注意：
        1. **鎖定區域**：PDF 中可能包含全台灣的資料，請只針對「{selected_region}」的氣溫與天氣狀況進行分析。
        2. **分辨日期**：請準確抓出預報中的日期與星期。
        3. **時尚建議**：請根據該區域每一天的天氣變化（轉涼、下雨、回溫），給出具體的穿搭主題與單品建議。
           (例如：週一有雨 -> 推薦英倫風風衣配雨靴；週三轉晴 -> 推薦亮色系亞麻襯衫)
        4. **格式要求**：只輸出 CSV 格式，包含標頭。

        CSV 欄位如下：
        "日期/星期","地區","天氣概況(氣溫)","當日時尚主題","穿搭單品推薦","必備配件"
        """
        # ==============================================================================

        # 3. 呼叫 Gemini (Flash 模型速度快適合讀長文件)
        model = genai.GenerativeModel('gemini-2.5-flash') # 修正模型名稱寫法，通常是 1.5-flash
        response = model.generate_content([prompt_text, gemini_file])

        genai.delete_file(gemini_file.name) # 清理

        # 4. 解析 CSV
        csv_output = response.text.strip().replace("```csv\n", "").replace("```", "")

        local_csv_name = "weekly_fashion.csv"
        with open(local_csv_name, "w", encoding='utf-8') as f:
            f.write(csv_output)

        df_result = pd.read_csv(local_csv_name)
        df_result = df_result.fillna('')

        status_msg += "✅ 一週穿搭分析完成！\n"

        # 5. 寫入 Google Sheets
        try:
            gsheets = gc.open_by_url(SHEET_URL)
            try:
                worksheet = gsheets.worksheet(WORKSHEET_NAME)
                worksheet.clear() # 先清空舊的一週資料
            except gspread.exceptions.WorksheetNotFound:
                worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

            # 重新寫入 Header 與 Data
            set_with_dataframe(worksheet, df_result)
            status_msg += f"✅ 資料已同步至 Google Sheet: {WORKSHEET_NAME}\n"

        except Exception as e:
             status_msg += f"⚠️ Google Sheet 寫入失敗: {e}\n"

        print(status_msg)
        return df_result

    except Exception as e:
        print(f"執行錯誤: {e}")
        if 'gemini_file' in locals(): genai.delete_file(gemini_file.name)
        return pd.DataFrame({"狀態": ["❌ 錯誤"], "詳情": [str(e)]})

In [46]:
# ---------------------------------------------------------
# Gradio 介面建置 (已修改：加入 Dropdown)
# ---------------------------------------------------------
with gr.Blocks(title="WeatherFit", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🌤️ WeatherFit")
    gr.Markdown("# AI天氣穿搭系統")
    gr.Markdown("WeatherFit自動抓取氣象局「一週預報」，為您規劃未來 7 天的穿搭行事曆。")

    with gr.Row():
        with gr.Column(scale=1):
            # 加入下拉選單
            region_input = gr.Dropdown(
                choices=["北部地區", "中部地區", "南部地區", "東部地區", "外島地區"],
                value="北部地區", # 預設值
                label="請選擇您的區域"
            )
            run_btn = gr.Button("🔮 生成一週穿搭計畫", variant="primary", size="lg")

        with gr.Column(scale=3):
            gr.Markdown("系統將分析一週氣溫變化，這可能需要 20-30 秒，請稍候...")

    output_table = gr.Dataframe(label="未來一週穿搭建議", interactive=False, wrap=True)

    # 按鈕事件綁定
    run_btn.click(
        fn=get_weekly_fashion_advice,
        inputs=region_input,  # 這裡把下拉選單的值傳進去
        outputs=output_table
    )

/tmp/ipython-input-2840215647.py:4: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="WeatherFit", theme=gr.themes.Soft()) as demo:


In [ ]:
# 啟動 Gradio
# debug=True 方便在 Colab 中看到錯誤訊息
demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://61e1d4a2157071d4b1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🚀 開始執行 [北部地區] 的一週穿搭分析...

正在連線至 https://www.cwa.gov.tw/V8/C/W/pdf.html 尋找1週預報...
✅ 找到目標連結文字：1週天氣PDF
正在下載 PDF: https://www.cwa.gov.tw/Data/fcst_pdf/W11.pdf
🚀 開始執行 [北部地區] 的一週穿搭分析...
✅ PDF 下載成功，AI 正在規劃一週穿搭...
✅ 一週穿搭分析完成！
✅ 資料已同步至 Google Sheet: 工作表1



In [ ]:
'''import google.generativeai as genai
from google.colab import userdata

# 重新設定一次 Key (確保環境變數正確)
api_key = userdata.get('HW3')
genai.configure(api_key=api_key)

print("正在查詢可用的模型清單...")
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)'''